In [1]:
import keras as K
import os.path

import sys

sys.path.append("../src")
from artstat import util
import numpy as np

Using TensorFlow backend.


In [2]:
import imp
imp.reload(util)

<module 'artstat.util' from '../src/artstat/util.py'>

In [3]:
datadir = "/home/pmilovanov/hg/my/data/gallery-pr/all"

path_train = os.path.join(datadir, "train")
path_test = os.path.join(datadir, "test")

glove = "/home/pmilovanov/data/glove/glove.840B.300d.txt"

In [4]:
words, vocab = util.load_vocab("../vocab.txt", 100000)

In [5]:
emb_matrix = util.load_embeddings(vocab, 300, glove)

In [7]:
print(emb_matrix.shape, len(vocab))

(100001, 300) 100000


In [8]:
imp.reload(util)

seqlen = 128
stride = 96
X, Y, Xu, Yu = util.load_data_sequences(path_train, vocab, seqlen, stride)


100%|##########| 22860/22860 [00:35<00:00, 652.87it/s]


In [77]:
Yfake = np.zeros((Yu.shape[0], 128), dtype="float32")


In [26]:
from keras.layers import Dense, Embedding, Reshape, Concatenate, CuDNNLSTM, Lambda
import keras as K
from keras import backend as B
import tensorflow as tf

In [10]:
input_x = K.Input((seqlen,), dtype="int32", name="input_x")
input_xu = K.Input((seqlen,), dtype="float32", name="input_xu")
input_y = K.Input((seqlen,), dtype="int32", name="input_y")
input_yu = K.Input((seqlen,), dtype="float32", name="input_yu")

reshaper_u  = K.layers.Reshape((seqlen,1))
resh_xu = reshaper_u(input_xu)
resh_yu = reshaper_u(input_yu)

dim = emb_matrix.shape[1] + 1

L_emb = K.layers.Embedding(emb_matrix.shape[0], 
                           emb_matrix.shape[1], 
                           input_length=seqlen, 
                          trainable=False,
                           weights=[emb_matrix],
                           name="embedding")


In [11]:
Lemb = L_emb
emb_x = Lemb(input_x)
emb_y = Lemb(input_y)

In [12]:
Lconcat = K.layers.Concatenate()
concat_x = Lconcat([emb_x, resh_xu])
concat_y = Lconcat([emb_y, resh_yu])

In [23]:
def realdot(a,b):
    d = K.layers.multiply([a, b])
    return B.sum(d, axis=-1)

def dotlayer(x):
    a,b = x
    return realdot(a,b)

def cossim(x):
    a,b = x
    p = realdot(a,b)
    an = B.sqrt(realdot(a,a) + B.epsilon())
    bn = B.sqrt(realdot(b,b) + B.epsilon())
    return tf.divide(p, K.layers.multiply([an, bn]) + B.epsilon())

In [92]:
lstm =  K.layers.CuDNNLSTM(256, return_sequences=True, name='lstm1')(concat_x)
lstm = K.layers.BatchNormalization()(lstm)
lstm =  K.layers.CuDNNLSTM(256, return_sequences=True, name='lstm2')(lstm)
#lstm = K.layers.CuDNNLSTM(256, return_sequences=True)(lstm)
dense = K.layers.BatchNormalization()(lstm)

def candidate_gen(inp, layername):
    out = K.layers.Dense(256, activation="relu", name=layername+"_1")(inp)
    out = K.layers.BatchNormalization()(out)
    out = K.layers.Dense(256, activation="relu", name=layername+"_2")(out)
    out = K.layers.BatchNormalization()(out)
    out = K.layers.Dense(dim, activation="sigmoid", name=layername+"_last")(out)
    return out

out1 = candidate_gen(dense, "out1")
out2 = candidate_gen(dense, "out2")
out3 = candidate_gen(dense, "out3")

allout = K.layers.Concatenate(name="concat_out")([out1, out2, out3, lstm])
probs = K.layers.Dense(256, activation="relu", name="probabilities_intermediate")(allout)
probs = K.layers.BatchNormalization()(probs)
probs = K.layers.Dense(3, activation="softmax", name="probabilities")(probs)

def fexpander(x):
    return B.expand_dims(x)
L_expander = Lambda(fexpander, name="expander")

L_cossim = Lambda(cossim, name="cosine_similarity")
prox1 = L_cossim([concat_y, out1])
prox1 = L_expander(prox1)
prox2 = L_cossim([concat_y, out2])
prox2 = L_expander(prox2)
prox3 = L_cossim([concat_y, out3])
prox3 = L_expander(prox3)

allprox = Concatenate(name="concat_prox")([prox1, prox2, prox3])
final1 = K.layers.Lambda(dotlayer, name="final1")([probs, allprox])

#allones = B.ones(final1.shape, dtype="float32")
def final2_fun(x):
    return B.abs(1.0 - x)

final2 = Lambda(final2_fun, name="final2")(final1)
print(probs.shape)
print(prox3.shape)
print(allout.shape)
print(allprox.shape)
print(final1.shape)
print(final2.shape)




(?, 128, 3)
(?, 128, 1)
(?, 128, 1159)
(?, 128, 3)
(?, 128)
(?, 128)


In [93]:
model = K.Model(inputs=[input_x, input_xu, input_y, input_yu],
               outputs=[final2])

In [94]:
model.summary()


__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_x (InputLayer)            (None, 128)          0                                            
__________________________________________________________________________________________________
input_xu (InputLayer)           (None, 128)          0                                            
__________________________________________________________________________________________________
embedding (Embedding)           (None, 128, 300)     30000300    input_x[0][0]                    
                                                                 input_y[0][0]                    
__________________________________________________________________________________________________
reshape_1 (Reshape)             (None, 128, 1)       0           input_xu[0][0]                   
          

In [95]:
opt = K.optimizers.Adam(lr=0.01)
model.compile(opt, loss='mean_absolute_error')

In [ ]:
model.fit([X, Xu, Y, Yu], [Yfake], batch_size=1024, epochs=1000000)


Epoch 1/1000000
 31744/157646 [=====>........................] - ETA: 1:04 - loss: 0.5561

In [97]:
predictor = K.Model([input_x, input_xu], [out1, out2, out3, probs])
predictor.compile(opt, loss="mean_absolute_error")

In [98]:
from sklearn import preprocessing as pp
norm_emb_matrix =  pp.normalize(Lemb.get_weights()[0], axis=1)

In [99]:
len(X)

157646

In [100]:
i = 0
gen = X[i].tolist()
genu = Xu[i].tolist()

tX = np.zeros((1, seqlen), dtype="int32")
tXu = np.zeros((1, seqlen), dtype="float32")
results = []

def vec2word(x):
    we = np.transpose(pp.normalize(np.reshape(x[0, -1, :300], (1,300))))
    idx = np.argmax(np.matmul(norm_emb_matrix, we))
    return idx

iterations = 128
for j in range(iterations):
    tX[0,:] = np.array(gen[-seqlen:], "int32")
    tXu[0,:] = np.array(genu[-seqlen:], "float32")
    #print(tX)
    xv, xuv = 0, 0.0
    yhat = predictor.predict([tX, tXu])
    
    #for qq in yhat:
    #    print(qq.shape)
    #print(yhat)
    #break
    vecid = np.random.choice(3, p=yhat[3][0,0])
    z = yhat[vecid]
    
    candidates=[]
    for u,c in enumerate(yhat[:3]):
        cid = vec2word(c)
        candidates.append((words[cid], yhat[3][0,0,u]))
    print(candidates)
    
    if False: #z[0, -1, -1] > 0.9:
        xuv = 1.0
        results.append("<UNK>")
    else:
        we = np.transpose(pp.normalize(np.reshape(z[0, -1, :300], (1,300))))
#        print(np.linalg.norm(we))
#        scores = np.matmul(norm_emb_matrix, we)
#        print(scores)
#        print(np.min(scores))
#        break
        xv = np.argmax(np.matmul(norm_emb_matrix, we))
        if xv:
            results.append(words[xv])
        else:
            results.append("<EMPTY>")
    gen.append(xv)
    genu.append(xuv)
    print(results[-1])

[('the', 0.0), ('registrations', 1.0), ('see', 1.2173728e-19)]
registrations
[('as', 2.4678491e-15), (',', 1.0), ('the', 2.4923575e-11)]
,
[('that', 0.0), ('and', 1.0), ('see', 4.6645018e-24)]
and
[('that', 0.9999981), ('or', 1.2818558e-06), ('see', 5.48748e-07)]
that
[('that', 0.9999987), ('30', 7.297992e-07), ('see', 5.628823e-07)]
that
[('that', 0.9999975), ('and', 2.152433e-06), ('see', 3.671109e-07)]
that
[('that', 0.99999917), ('and', 4.184265e-07), ('see', 3.151159e-07)]
that
[('that', 0.9999989), ('as', 6.550726e-07), ('see', 4.4266216e-07)]
that
[('that', 0.9999993), ('as', 4.1415322e-07), ('see', 3.79203e-07)]
that
[('that', 0.99984264), ('as', 0.000150216), ('see', 7.0977485e-06)]
that
[('that', 0.0), ('as', 1.0), ('see', 2.1378512e-20)]
as
[('that', 0.99999833), ('during', 9.854743e-07), ('see', 6.876204e-07)]
that
[('that', 0.9999962), ('and', 3.1465715e-06), ('see', 6.9511685e-07)]
that
[('that', 0.99999654), ('or', 2.7956905e-06), ('see', 6.943386e-07)]
that
[('that', 0.

[('that', 0.9999968), ('as', 2.2640047e-06), ('see', 9.542453e-07)]
that
[('that', 0.9999993), ('as', 3.981528e-07), ('see', 3.427529e-07)]
that
[('that', 0.999995), ('as', 3.635814e-06), ('see', 1.4607718e-06)]
that
[('that', 0.9999939), ('as', 4.976207e-06), ('see', 1.0990798e-06)]
that
[('that', 0.99999905), ('as', 7.3650676e-07), ('see', 2.6873386e-07)]
that
[('that', 0.9999987), ('as', 7.297992e-07), ('see', 5.628823e-07)]
that
[('that', 0.99984264), ('as', 0.000150216), ('see', 7.0977485e-06)]
that
[('that', 0.0), ('as', 1.0), ('see', 2.1378512e-20)]
as
[('that', 0.99999833), ('during', 9.854743e-07), ('see', 6.876204e-07)]
that
[('that', 0.9999802), ('as', 1.831076e-05), ('see', 1.4841868e-06)]
that
[('that', 0.99994946), ('as', 4.726328e-05), ('see', 3.3599729e-06)]
that


In [66]:
print(np.min(z), np.max(z))

1.3547268e-16 0.93638355


In [92]:
from sklearn import preprocessing

In [93]:
norm_emb_matrix =  preprocessing.normalize(emb_matrix, axis=1)

In [94]:
zn = preprocessing.normalize(z[0,:], axis=1)


In [95]:
we = zn[-1, :300]

In [96]:
scores = np.matmul(norm_emb_matrix, we)

In [97]:
idx = np.argmax(scores)
print(idx)
word = words[idx]

23


In [98]:
word

'at'

In [84]:
words[2]

'.'